Identify **port scanning** - technique used to find network hosts that have services listening on one or more target ports

In **SYN scan**, the scanner sends **TCP SYN** packets (the first packet in
the TCP handshake) and watches for hosts that respond with **SYN+ACK** packets (the second
handshake step). Since most hosts are not prepared to receive connections on any given
port, typically, during a port scan, a much smaller number of hosts will respond with
**SYN+ACK** packets than originally received **SYN** packets.

By observing this effect in a packet
trace, you can identify source addresses that may be attempting a port scan. Your task is to
develop a Python program that analyzes a pcap file in order to detect possible **SYN scans**.

--------------

The output should be the set of IP addresses (one per line) that (1) sent more than 3 times
as many SYN packets as the number of SYN+ACK packets they received and (2) sent out at
least one SYN packet but has not received any SYN+ACK packets. Your program should
silently ignore packets that are malformed or that are not using Ethernet, IP, and TCP.

In [12]:
import binascii
import dpkt

f = open('com402_hw6_ex3_p2.pcap', 'rb')
pcap = dpkt.pcap.Reader(f)

ip_sent_syn = {}
ip_received_synack = {}

# Iterate through all packets
for ts, buf in pcap:
    eth = dpkt.ethernet.Ethernet(buf)
    ip = eth.data
    tcp = ip.data
    
    # Get src and dst IP addresses
    ip_src = ip.src
    ip_dst = ip.dst
    eth_src = eth.src
    eth_dst = eth.dst
    
    # Get SYN and ACK flags
    try:
        syn_flag = (tcp.flags & dpkt.tcp.TH_SYN) != 0
        ack_flag = (tcp.flags & dpkt.tcp.TH_ACK) != 0
        
        if syn_flag:
            if not ack_flag:
                # Save SYN packets
                if ip_src in ip_sent_syn.keys():
                    ip_sent_syn[ip_src] += 1
                else:
                    ip_sent_syn[ip_src] = 1
            else:
                # Save SYN+ACK packets
                if ip_dst in ip_received_synack.keys():
                    ip_received_synack[ip_dst] += 1
                else:
                    ip_received_synack[ip_dst] = 1
    except AttributeError:
        pass

# Get suspicious IPs
ip_suspicious = []

for src, nb_syn in ip_sent_syn.items():
    if src in ip_received_synack.keys():
        nb_synack = ip_received_synack[src]
        if nb_syn > 3 * nb_synack:
            ip_suspicious.append(src)
    else:
        ip_suspicious.append(src)

# Print solution
for ip_add in ip_suspicious:
    ip_add = str(binascii.hexlify(ip_add))[2:-1]
    num_ip_add = [int(ip_add[i:i+2], 16) for i in range(0, len(ip_add), 2)]
    print('{}.{}.{}.{}'.format(num_ip_add[0], num_ip_add[1], num_ip_add[2], num_ip_add[3]))